# Entrega Final
## Limpieza y preprocesado y creación de modelo Regresion Logística
Este notebook contiene una exploración básica, operaciones de limpieza y prepocesado de datos, asi como la creacion del modelo de Regresion Logística.

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
from google.colab import files

files.upload()

In [ ]:
# cargar archivo kaggle.json
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!chmod 600 /content/kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

In [ ]:
# descomprimir archivos
!unzip udea*.zip > /dev/null
!wc *.csv

In [ ]:
# Cargar archivo train.csv
file = "train.csv"
df_train = pd.read_csv(file)
df_train.head()

## Exploración de datos
### Número de estudiantes por programa académico

In [ ]:
# Cantidad de estudiantes por programa
df_estudiantes_por_programa = df_train["ESTU_PRGM_ACADEMICO"].value_counts()
df_estudiantes_por_programa.head()

### Número de estudiantes por departamento

In [ ]:
# Cantidad de estudiantes por departamento
df_estudiantes_por_departamento = df_train["ESTU_PRGM_DEPARTAMENTO"].value_counts()
df_estudiantes_por_departamento.head()

### Gráfico de horas semanales que trabajan los estudiantes

In [ ]:
# Cantidad de horas que trabajan los estudiantes a la semana
df_trabajo_por_semana_estudiantes = df_train["ESTU_HORASSEMANATRABAJA"].value_counts()

# Crear gráfico
fig, ax = plt.subplots(figsize=(10, 6))
df_trabajo_por_semana_estudiantes.plot(kind="bar", ax=ax, color="firebrick")
plt.title("Horas semanales trabajadas por Estudiantes", fontsize=16)
plt.xlabel("Cantidad de Horas", fontsize=12)
plt.ylabel("Cantidad de Estudiantes", fontsize=12)
plt.xticks(rotation=75)
plt.show()

### Gráfico de comparativa entre el valor de la matrícula del estudiante y el estrato de vivienda familiar

In [ ]:
# Agrupar datos por valor de matrícula y estrato socioeconómico, y contar la cantidad de estudiantes
data_agrupada = (
    df_train.groupby(["ESTU_VALORMATRICULAUNIVERSIDAD", "FAMI_ESTRATOVIVIENDA"])
    .size()
    .unstack()
)

# Crear gráfico
fig, ax = plt.subplots(figsize=(10, 6))
data_agrupada.plot(kind="bar", ax=ax)
plt.title("Valor de Matrícula de Estudiante según Estrato Socioeconómico", fontsize=16)
plt.xlabel("Valor Matrícula Estudiante", fontsize=12)
plt.ylabel("Cantidad de Estudiantes", fontsize=12)
plt.xticks(rotation=75)
plt.legend(title="Estrato Socioeconómico")
plt.show()

## Preprocesado de datos
### Imputación de Datos Faltantes

In [ ]:
# Separar las columnas numéricas y categóricas
columnas_numericas = df_train.select_dtypes(include=["float64", "int64"]).columns
columnas_categoricas = df_train.select_dtypes(include=["object"]).columns

# Imputar columnas numéricas con la media (promedio)
imputar_numericas = SimpleImputer(strategy="mean")
df_train[columnas_numericas] = imputar_numericas.fit_transform(
    df_train[columnas_numericas]
)

# Imputar columnas categóricas con la moda (más frecuente)
imputar_categoricas = SimpleImputer(strategy="most_frequent")
df_train[columnas_categoricas] = imputar_categoricas.fit_transform(
    df_train[columnas_categoricas]
)

# Verificar si quedan valores faltantes
print("Valores faltantes después de la imputación:")
print(df_train.isnull().sum())

### Normalización de Valores Numéricos

In [ ]:
# Crear un objeto MinMaxScaler
scaler = MinMaxScaler()

# Aplicar la normalización a las columnas numéricas
df_train[columnas_numericas] = scaler.fit_transform(df_train[columnas_numericas])

# Verificar los datos normalizados
print("Datos normalizados:")
print(df_train[columnas_numericas].head())

### Detección y Eliminación de Valores Atípicos (Outliers) en `ESTU_HORASSEMANATRABAJA`

In [ ]:
# inicializar variable que contiene columna
column = "ESTU_HORASSEMANATRABAJA"

In [ ]:
# Convertir la columna 'ESTU_HORASSEMANATRABAJA' a numérico, forzando valores no numéricos a NaN
df_train[column] = pd.to_numeric(df_train[column], errors="coerce")

# Verificar si hay valores NaN después de la conversión
print(f"Valores faltantes en '{column}' después de la conversión:")
print(df_train[column].isnull().sum())

# Llenar valores faltantes con la media (promedio)
df_train[column].fillna(df_train[column].mean(), inplace=True)

In [ ]:
# Calcular los límites para detectar outliers usando el IQR en la columna 'ESTU_HORASSEMANATRABAJA'
Q1 = df_train[column].quantile(0.25)
Q3 = df_train[column].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites inferior y superior
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar el DataFrame eliminando los outliers
df_sin_outliers = df_train[
    (df_train[column] >= limite_inferior) & (df_train[column] <= limite_superior)
]

# Verificar cuántos valores se eliminaron
print(f"Valores originales: {len(df_train)}")
print(f"Valores después de eliminar outliers: {len(df_sin_outliers)}")

if len(df_train) == len(df_sin_outliers):
    print(f"No se encontraron outliers en la columna '{column}'")

# Ver los primeros valores después de eliminar los outliers
print(df_sin_outliers.head())

In [ ]:
# Reducción de memoria: cambiar el tipo de columnas numéricas
for col in df_train.select_dtypes(include=["int", "float"]).columns:
    df_train[col] = pd.to_numeric(df_train[col], downcast="integer")

In [ ]:
# Codificación de variables categóricas
label_encoder = LabelEncoder()
df_train["RENDIMIENTO_GLOBAL"] = label_encoder.fit_transform(
    df_train["RENDIMIENTO_GLOBAL"]
)
for col in df_train.select_dtypes(include=["object"]).columns:
    df_train[col] = label_encoder.fit_transform(df_train[col])

## Creación de modelo Regresion Logística
### Dividir variables X y Y en entrenamiento y prueba

In [ ]:
# Separar variables predictoras y objetivo
X = df_train.drop(columns=["RENDIMIENTO_GLOBAL", "ID"])
y = df_train["RENDIMIENTO_GLOBAL"]

In [ ]:
# Dividir el conjunto de datos para validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### Definición de modelo Regresion Logística

In [ ]:
# Creacion modelo Regresion logistica
modelo_rl = LogisticRegression(max_iter=200, random_state=42)
modelo_rl.fit(X_train, y_train)
y_pred_rl = modelo_rl.predict(X_val)
precision_rl = accuracy_score(y_val, y_pred_rl)

print("Precisión con Regresión Logística:", precision_rl)

## Aplicar procedimiento a datos de prueba

In [ ]:
file = "test.csv"
df_test = pd.read_csv(file)

### Imputar valores faltantes a df_test

In [ ]:
# Imputar valores faltantes en `df_test`
df_test[columnas_numericas] = imputar_numericas.transform(df_test[columnas_numericas])
df_test[columnas_categoricas] = imputar_categoricas.transform(
    df_test[columnas_categoricas]
)

### Normalización de valores

In [ ]:
# Normalizar las columnas numéricas en `df_test`
df_test[columnas_numericas] = scaler.transform(df_test[columnas_numericas])

### Codificación de variables categóricas

In [ ]:
# Codificación de variables categóricas en `df_test`
for col in df_test.select_dtypes(include=["object"]).columns:
    df_test[col] = label_encoder.transform(df_test[col])

### Predicción de los datos de prueba con el modelo creado

In [ ]:
# Predicción en los datos de prueba
X_test = df_test.drop(columns=["ID"])
predicciones = modelo_rl.predict(X_test)

### Crear dataframe para guardar predicción

In [ ]:
df_submission = pd.DataFrame({"ID": df_test["ID"], "RENDIMIENTO_GLOBAL": predicciones})

### Crear archivo .csv

In [ ]:
# Guardar el archivo de envío en formato CSV
df_submission.to_csv("submission.csv", index=False)
print("Archivo 'submission.csv' creado con éxito.")

In [ ]:
df_submission.shape

In [ ]:
!head submission_optimized.csv

### Enviar archivo .csv a la competencia

In [ ]:
!kaggle competitions submit -c udea-ai4eng-20242 -f submission.csv -m "Manuela Sanchez Arenas modelo Regresion Logística"